In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from tensorflow.python.ops.distributions.categorical import Categorical
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

### 데이터 읽어오기

In [ ]:
def extract(df):
    X = df.drop(columns=['MEDV'])
    Y = df['MEDV']
    return X, Y


def readCSV(fileName):
    df = (pd.read_csv(fileName,
                      dtype={'CHAS': 'bool',
                             'TAX': 'int'})
          .set_index('ID'))
    return df


# 데이터 읽기
data = readCSV('data/train.csv')
test = readCSV('data/test.csv')

# 데이터 전처리
data = data[data['MEDV'] != 50.0]

# 데이터 전달
input_x, input_y = extract(data)
submit_x = test

### 검증

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# 검증
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.8, colsample_bytree=0.8, random_state=42)
rmse_scores = -cross_val_score(model, input_x, input_y, scoring='neg_root_mean_squared_error', cv=5)

# 평균 RMSE 출력
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

### 훈련 및 예측 파일 생성

In [ ]:
# 훈련
model.fit(input_x, input_y)

# 결과 파일 생성
submit_y = pd.DataFrame(model.predict(submit_x), index=submit_x.index, columns=['MEDV'])
submit_y.to_csv("submission.csv", index=True)